In [4]:

pip install flask pandas requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from flask import Flask, jsonify, request
import pandas as pd
import requests

app = Flask(__name__)

# Step 3: Fetch the data from the third-party API and initialize the database (in-memory)
DATA_URL = "https://s3.amazonaws.com/roxiler.com/product_transaction.json"
response = requests.get(DATA_URL)
data = response.json()

# Convert the JSON data into a pandas DataFrame
df = pd.DataFrame(data)

# Convert the dateOfSale to datetime to extract the month later
df['dateOfSale'] = pd.to_datetime(df['dateOfSale'])

# Helper function to filter by month
def filter_by_month(df, month):
    return df[df['dateOfSale'].dt.month == month]

# Step 4: Create the API to list all transactions (with search and pagination)
@app.route('/transactions', methods=['GET'])
def get_transactions():
    month = int(request.args.get('month', 1))  # Default to January
    search = request.args.get('search', '').lower()
    page = int(request.args.get('page', 1))
    per_page = int(request.args.get('per_page', 10))

    # Filter by month
    filtered_df = filter_by_month(df, month)

    # Apply search filters if provided
    if search:
        filtered_df = filtered_df[
            filtered_df['title'].str.lower().str.contains(search) |
            filtered_df['description'].str.lower().str.contains(search) |
            filtered_df['price'].astype(str).str.contains(search)
        ]

    # Pagination
    start = (page - 1) * per_page
    end = start + per_page
    paginated_df = filtered_df.iloc[start:end]

    # Convert the DataFrame to a list of dictionaries (JSON format)
    transactions = paginated_df.to_dict(orient='records')
    return jsonify(transactions)

# Step 5: Create the API for statistics
@app.route('/statistics', methods=['GET'])
def get_statistics():
    month = int(request.args.get('month', 1))  # Default to January
    filtered_df = filter_by_month(df, month)

    total_sale_amount = filtered_df['price'].sum()
    total_sold_items = len(filtered_df[filtered_df['sold'] == True])
    total_not_sold_items = len(filtered_df[filtered_df['sold'] == False])

    stats = {
        "totalSaleAmount": total_sale_amount,
        "totalSoldItems": total_sold_items,
        "totalNotSoldItems": total_not_sold_items
    }
    return jsonify(stats)

# Step 6: Create the API for bar chart (price range)
@app.route('/bar-chart', methods=['GET'])
def get_bar_chart():
    month = int(request.args.get('month', 1))  # Default to January
    filtered_df = filter_by_month(df, month)

    # Define price ranges
    ranges = [(0, 100), (101, 200), (201, 300), (301, 400), (401, 500), 
              (501, 600), (601, 700), (701, 800), (801, 900), (901, float('inf'))]

    # Count the number of items in each price range
    bar_chart_data = []
    for lower, upper in ranges:
        count = len(filtered_df[(filtered_df['price'] >= lower) & (filtered_df['price'] <= upper)])
        bar_chart_data.append({
            "range": f"{lower}-{upper}" if upper != float('inf') else "901+",
            "count": count
        })

    return jsonify(bar_chart_data)

# Step 7: Create the API for pie chart (category distribution)
@app.route('/pie-chart', methods=['GET'])
def get_pie_chart():
    month = int(request.args.get('month', 1))  # Default to January
    filtered_df = filter_by_month(df, month)

    # Get unique categories and their item counts
    pie_chart_data = filtered_df['category'].value_counts().to_dict()
    return jsonify(pie_chart_data)

# Step 8: Create the combined API for all data
@app.route('/combined-data', methods=['GET'])
def get_combined_data():
    month = int(request.args.get('month', 1))  # Default to January
    
    # Fetch transactions, statistics, bar chart, and pie chart data
    transactions = get_transactions().json
    statistics = get_statistics().json
    bar_chart = get_bar_chart().json
    pie_chart = get_pie_chart().json

    # Combine all the data into one response
    combined_data = {
        "transactions": transactions,
        "statistics": statistics,
        "bar_chart": bar_chart,
        "pie_chart": pie_chart
    }
    return jsonify(combined_data)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\prajw\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
